In [1]:
stock = 'TWTR'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    43.419998
2018-07-23    43.310001
2018-07-24    42.169998
2018-07-25    44.220001
2018-07-26    42.939999
2018-07-27    34.119999
2018-07-30    31.379999
2018-07-31    31.870001
2018-08-01    31.910000
2018-08-02    32.820000
2018-08-03    31.959999
2018-08-06    32.980000
2018-08-07    32.669998
2018-08-08    31.840000
2018-08-09    31.959999
2018-08-10    32.009998
2018-08-13    32.799999
2018-08-14    33.189999
2018-08-15    32.380001
2018-08-16    32.830002
2018-08-17    32.730000
2018-08-20    32.599998
2018-08-21    33.689999
2018-08-22    33.810001
2018-08-23    33.880001
2018-08-24    34.279999
2018-08-27    35.889999
2018-08-28    35.490002
2018-08-29    35.349998
2018-08-30    35.639999
                ...    
2018-09-17    28.860001
2018-09-18    29.219999
2018-09-19    29.520000
2018-09-20    29.850000
2018-09-21    28.500000
2018-09-24    28.600000
2018-09-25    29.110001
2018-09-26    29.010000
2018-09-27    29.420000
2018-09-28    28.459999
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


28.509089818797953
[28.99432493 27.5769     28.57453125 27.50675066 29.89294225]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")